In [27]:
import pandas as pd
import requests
from collections import defaultdict
from math import radians, sin, cos, sqrt, atan2

In [2]:
# StopTimes
# trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled
df_stop_times = pd.read_csv("gtfs-data/stop_times.txt")
df_stop_times["stop_id"] = df_stop_times["stop_id"].astype(int)
# print(df_stop_times.head())

In [3]:
# Trips
# route_id,service_id,trip_id,direction_id,block_id,shape_id,direction,wheelchair_accessible,schd_trip_id
df_trips = pd.read_csv("gtfs-data/trips.txt")
# print(df_trips.head())

In [4]:
# Routes
# route_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
df_routes = pd.read_csv("gtfs-data/routes.txt")
# print(df_routes.head())

In [5]:
# Stops
# stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
df_stops = pd.read_csv("gtfs-data/stops.txt")
# print(df_stops.head())

In [6]:
# RouteStops - {route_id : ordered stops}
merged = df_stop_times.merge(df_trips[['trip_id', 'route_id']], on='trip_id', how='left')
merged = merged.sort_values(by=['route_id', 'trip_id', 'stop_sequence'])

RouteStops = (
    merged.groupby('route_id')['stop_id']
    .apply(lambda stops: list(dict.fromkeys(stops)))
    .to_dict()
)

# print first 3 routes 
# for route, stops in list(RouteStops.items())[:3]:  
#     print(f"Route {route}: {stops[:10]} ... ({len(stops)} stops total)")

In [7]:
# RouteTrips - {route_id : trips}
RouteTrips = (
    df_trips.groupby('route_id')['trip_id']
    .apply(list)
    .to_dict()
)

# print first 3 routes
# for route, trips in list(RouteTrips.items())[:3]:  
#     print(f"Route {route}: {trips[:10]} ... ({len(trips)} trips total)")

In [8]:
def gtfs_time_to_seconds(time_str):
    hours, mins, seconds = map(int, time_str.split(":"))
    return hours * 3600 + mins * 60 + seconds

In [9]:
# Trips - {trip_id : {stops : {stop_id : arrival_time, departure_time}}}
df_stop_times = df_stop_times.sort_values(by=["trip_id", "stop_sequence"])

trip_info_columns = [col for col in df_trips.columns if col != "trip_id"]

Trips = (
    df_trips.set_index("trip_id")[trip_info_columns]
    .to_dict(orient="index")
)

for trip_id, group in df_stop_times.groupby("trip_id"):
    stops_dict = {}
    for _, row in group.iterrows():
        stop_id = int(row["stop_id"])
        arrival = row["arrival_time"]
        departure = row["departure_time"]

        stops_dict[stop_id] = {
            "arrival_time": gtfs_time_to_seconds(arrival),
            "departure_time": gtfs_time_to_seconds(departure)
        }

    if trip_id in Trips:
        Trips[trip_id]["stops"] = stops_dict
    else:
        Trips[trip_id] = {"stops": stops_dict}

# for trip, trip_info in list(Trips.items())[:3]:
#     print(f"\nTrip {trip}:")
#     for k, v in trip_info.items():
#         if k == "stops":
#             # show first 3 stops (as dict preview)
#             subset = dict(list(v.items())[:3])
#             print(f"  {k}: {subset} ... ({len(v)} stops total)")
#         else:
#             print(f"  {k}: {v}")

In [10]:
# StopRoutes - {stop_id : routes}
merged = df_stop_times.merge(df_trips[['trip_id', 'route_id']], on='trip_id', how='left')

StopRoutes = (
    merged.groupby('stop_id')['route_id']
    .apply(lambda x: set(x.dropna()))
    .to_dict()
)

# print first 3 stops
# for stop, routes in list(StopRoutes.items())[:3]:
#     print(f"Stop {stop}: {routes}")

In [53]:
StopCoords = {}
for _, row in df_stops.iterrows():
    StopCoords[int(row['stop_id'])] = (row['stop_lat'], row['stop_lon'])

def get_walking_distance(lat1, lon1, lat2, lon2):
    R = 6371000
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [54]:
Transfers = defaultdict(list)

stop_ids = [sid for sid in StopCoords.keys()]

for i, stop1 in enumerate(stop_ids):
    lat1, lon1 = StopCoords[stop1]

    for stop2 in stop_ids[i+1:]:
        if stop1 == stop2:
            continue

        lat2, lon2 = StopCoords[stop2]
        distance = get_walking_distance(lat1, lon1, lat2, lon2)

        if distance <= 500:
            walking_time = int(distance / 1.4)
            Transfers[stop1].append((stop2, walking_time))
            Transfers[stop2].append((stop1, walking_time))

In [56]:
# Earliest trip in route r (route) that one can catch at stop p (next_stop)
def earliest_trip(route_id, board_stop, board_time):
    best_trip = None
    best_dep = float('inf')
    
    for trip_id in RouteTrips.get(route_id, []):
        stops = Trips[trip_id].get("stops", {})
        if board_stop not in stops:
            continue
        dep = stops[board_stop]["departure_time"]
        if dep >= board_time and dep < best_dep:
            best_dep = dep
            best_trip = trip_id
            
    return best_trip

In [95]:
def raptor(source_stop, dest_stop, departure_time, K):
    stop_arrival_times = {} 
    earliest_stop_arrival_times = {}
    
    # initializing the earliest arrival times to each stop to infinity
    for stop in StopCoords.keys():
        stop_arrival_times[stop] = [float('inf')] * (K + 1) # {stops : []*(K+1)}
        earliest_stop_arrival_times[stop] = float('inf') # {stop : arrival time}
    
    stop_arrival_times[source_stop][0] = departure_time # set initial stop arrival time to be departure time for the first round
    earliest_stop_arrival_times[source_stop] = departure_time
    route_taken = {} # (stop, k) -> (prev_stop, prev_k, trip_id or walk, walking time)
    
    # mark first stop
    marked_stops = [source_stop]
    
    for k in range(1, K + 1):
        Q = {} # route id: stop id
        
        # for each stop marked in the previous round
        for marked_stop in marked_stops:
            # iterate through all the routes that serve that stop
            for route_id in StopRoutes.get(marked_stop, []):
                # store the earliest marked stop with each route id
                marked_stop_idx = RouteStops[route_id].index(marked_stop)
                if route_id not in Q or marked_stop_idx < Q[route_id]:
                    Q[route_id] = marked_stop_idx
                    
        # unmark all stops       
        marked_stops.clear()
        
        # traverse each marked route and stop
        for route_id, stop_id in Q.items():
            current_trip = None
            boarding_stop = RouteStops[route_id][stop_id]
            boarding_time = stop_arrival_times[boarding_stop][k - 1]
            if boarding_time == float('inf'):
                continue
            
            current_trip = earliest_trip(route_id, boarding_stop, boarding_time)
            
            if current_trip is None:
                continue
            
            for next_stop in RouteStops[route_id][stop_id:]:
                if next_stop not in Trips[current_trip]['stops']:
                    continue
                    
                curr_trip_arr_time = Trips[current_trip]['stops'][next_stop]['arrival_time']
                curr_trip_dep_time = Trips[current_trip]['stops'][boarding_stop]['departure_time']
                
                if curr_trip_arr_time < curr_trip_dep_time:
                    continue
                    
                if curr_trip_arr_time < stop_arrival_times[next_stop][k]:
                    stop_arrival_times[next_stop][k] = curr_trip_arr_time
                    earliest_stop_arrival_times[next_stop] = min(earliest_stop_arrival_times[next_stop], curr_trip_arr_time)

                    route_taken[(next_stop, k)] = (boarding_stop, k - 1, current_trip, 0)

                    if next_stop not in marked_stops:
                        marked_stops.append(next_stop)
        
        marked_stops_temp = []
        
        for stop in marked_stops:
            for walkable_stop, walk_time in Transfers.get(stop, []):
                curr_walk_arr_time = stop_arrival_times[stop][k - 1] + walk_time
                
                if curr_walk_arr_time < stop_arrival_times[walkable_stop][k]:
                    stop_arrival_times[walkable_stop][k] = curr_walk_arr_time
                    
                    earliest_stop_arrival_times[walkable_stop] = min(earliest_stop_arrival_times[walkable_stop], curr_walk_arr_time)
                    
                    route_taken[(walkable_stop, k)] = (stop, k - 1, 'walk', walk_time)
                    
                    if walkable_stop not in marked_stops_temp:
                        marked_stops_temp.append(walkable_stop)
        
        marked_stops.extend(marked_stops_temp)

        if len(marked_stops) == 0:
            break
            
    best_time = earliest_stop_arrival_times.get(dest_stop, -1)
    if best_time == -1:
        print("No path found.")
        return None, None
    
    rounds_taken = None
    for k in range(K + 1):
        if stop_arrival_times[dest_stop][k] == best_time:
            rounds_taken = k
            break
        
    path = []
    curr_stop = dest_stop
    curr_round = rounds_taken
    
    while (curr_stop, curr_round) in route_taken:
        prev_stop, prev_round, mode, walk_time = route_taken[(curr_stop, curr_round)]
        
        if mode == 'walk':
            path.append({
                'type': 'walk',
                'stop1': prev_stop,
                'stop2': curr_stop,
                'walk_time': walk_time,
                'start_time': stop_arrival_times[prev_stop][prev_round],
                'end_time': stop_arrival_times[curr_stop][curr_round],
                'round': curr_round
            })
        else:
            path.append({
                'type': 'bus/train',
                'stop1': prev_stop,
                'stop2': curr_stop,
                'trip_id': mode,
                'start_time': Trips[mode]['stops'][prev_stop]['departure_time'],
                'end_time': Trips[mode]['stops'][curr_stop]['arrival_time'],
                'round': curr_round
            })
        
        curr_stop = prev_stop
        curr_round = prev_round
        
        if curr_round < 0:
            break
    
    path.reverse()
    
    return best_time, path

In [96]:
def seconds_to_hms(sec):
    h = sec // 3600
    m = (sec % 3600) // 60
    s = sec % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

In [97]:
print(Transfers[633])

[(543, 293), (544, 157), (545, 17), (547, 280), (631, 295), (634, 142), (636, 330), (717, 195), (794, 306), (796, 185), (797, 172), (798, 291), (3021, 330), (3023, 155), (3175, 146), (15043, 154), (15044, 128), (15861, 249), (17153, 244), (17154, 328), (18323, 309), (18642, 184), (18644, 328)]


In [98]:
print(Transfers[515])

[(516, 108), (662, 122), (11560, 303), (11561, 155), (11562, 13), (11563, 142), (11564, 287), (11586, 301), (11587, 155), (11589, 145), (11590, 285), (14512, 295), (14525, 282), (15298, 15), (15830, 300), (17368, 22), (18689, 160), (18720, 249), (18721, 117)]


In [99]:
print(Transfers[40920])
# we cannot go on a trip for any train line (no route associated for any train)
# still works for source or destination stop, will entail walking however

[(22, 284), (23, 258), (24, 305), (130, 301), (131, 265), (132, 302), (169, 315), (170, 165), (171, 71), (172, 155), (235, 190), (237, 132), (9036, 267), (9037, 126), (9039, 8), (9041, 213), (9084, 308), (9087, 39), (9089, 109), (14630, 271), (14837, 48), (15139, 296), (15553, 86), (17317, 232), (17328, 309), (30179, 0), (30180, 0)]


In [100]:
print(Transfers[40970])

[(159, 265), (160, 118), (161, 190), (162, 283), (243, 288), (244, 207), (246, 256), (3854, 241), (3855, 55), (3856, 129), (10249, 275), (10250, 203), (10251, 135), (10252, 6), (10253, 98), (10254, 242), (10283, 224), (10284, 98), (10285, 44), (10286, 115), (10287, 200), (10288, 262), (30187, 0), (30188, 0)]


In [101]:
print(RouteStops['Pink'])

[30113, 30082, 30117, 30028, 30151, 30201, 30086, 30143, 30040, 30161, 30199, 30032, 30295, 30221, 30074, 30050, 30384, 30132, 30166, 30031, 30007, 30141, 30222, 30296, 30033, 30200, 30162, 30041, 30144, 30087, 30202, 30152, 30029, 30118, 30083, 30114]


In [102]:
print(Transfers[30114])

[(6897, 174), (6900, 162), (6901, 151), (14193, 57), (30113, 0), (40580, 0)]


In [113]:
dep_time = 36000

arr_time, path = raptor(source_stop=30029, dest_stop=6897, departure_time=dep_time, K=5)
print("Departure time: ", seconds_to_hms(dep_time))
print("Arrival time: ", seconds_to_hms(arr_time))
print()

for i, transfer in enumerate(path, 1):
    if transfer['type'] == 'walk':
        print(f"{i} - WALK:")
        print(f"Walk from stop {transfer['stop1']} to stop {transfer['stop2']}")
        print(f"Start: {seconds_to_hms(transfer['start_time'])}, End: {seconds_to_hms(transfer['end_time'])}")
        print(f"Walking time: {transfer['walk_time']//60} min {transfer['walk_time']%60} s")
    else:
        print(f"{i} - BUS/TRAIN:")
        print(f"Board stop {transfer['stop1']}; Get down at stop {transfer['stop2']}")
        print(f"Start: {seconds_to_hms(transfer['start_time'])}, End: {seconds_to_hms(transfer['end_time'])}")
        transit_time = transfer['end_time'] - transfer['start_time']
        print(f"Transit time: {transit_time//60} min {transit_time%60} s")
        print()

Departure time:  10:00:00
Arrival time:  10:11:54

1 - BUS/TRAIN:
Board stop 30029, Get down at stop 30114
Start: 10:02:30, End: 10:09:00
Transit time: 6 min 30 s

2 - WALK:
Walk from stop 30114 to stop 6897
Start: 10:09:00, End: 10:11:54
Walking time: 2 min 54 s
